# ADS 509 Module 3: Group Comparison - Omar Elfeky

The task of comparing two groups of text is fundamental to textual analysis. There are innumerable applications: survey respondents from different segments of customers, speeches by different political parties, words used in Tweets by different constituencies, etc. In this assignment you will build code to effect comparisons between groups of text data, using the ideas learned in reading and lecture.

This assignment asks you to analyze the lyrics and Twitter descriptions for the two artists you selected in Module 1. If the results from that pull were not to your liking, you are welcome to use the zipped data from the “Assignment Materials” section. Specifically, you are asked to do the following: 

* Read in the data, normalize the text, and tokenize it. When you tokenize your Twitter descriptions, keep hashtags and emojis in your token set. 
* Calculate descriptive statistics on the two sets of lyrics and compare the results. 
* For each of the four corpora, find the words that are unique to that corpus. 
* Build word clouds for all four corpora. 

Each one of the analyses has a section dedicated to it below. Before beginning the analysis there is a section for you to read in the data and do your cleaning (tokenization and normalization). 


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


In [1]:
import os
import re
import emoji
import pandas as pd

from collections import Counter, defaultdict
from nltk.corpus import stopwords
from string import punctuation
from wordcloud import WordCloud 

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer


In [2]:
# Use this space for any additional import statements you need
from lexicalrichness import LexicalRichness
import csv
#default is 1e6
#jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10
#for my desktop
#os.environ['NUMEXPR_MAX_THREADS'] = '24'


In [3]:
# Place any addtional functions or constants you need here. 

# Some punctuation variations
punctuation = set(punctuation) # speeds up comparison
tw_punct = punctuation - {"#"}

# Stopwords
sw = stopwords.words("english")

# Two useful regex
whitespace_pattern = re.compile(r"\s+") #re.findall("[\w']+", text)
hashtag_pattern = re.compile(r"^#[0-9a-zA-Z]+")

# It's handy to have a full set of emojis
all_language_emojis = set()

for country in emoji.UNICODE_EMOJI : 
    for em in emoji.UNICODE_EMOJI[country] : 
        all_language_emojis.add(em)

# and now our functions
def descriptive_stats(tokens, num_tokens = 5, verbose=True) :
    """
        Given a list of tokens, print number of tokens, number of unique tokens, 
        number of characters, lexical diversity, and num_tokens most common
        tokens. Return a list of 
    """
    from collections import Counter

    # Fill in the correct values here. 
    num_tokens = len(tokens)
    num_unique_tokens = len(set().union(*tokens)) #set().union(*lyrics_data['tokens'][0])

    lexical_diversity = 0.0
    # Return Measure of Textual Lexical Diversity (MTLD).
    space = " "
    text = space.join( tokens )
    lex = LexicalRichness(text)
    #len(tokens)
    lexical_diversity = lex.hdd(draws=13) #lex.mtld(threshold=0.69) #lex.Herdan #lex.Summer 
    num_characters = 0
    for token in tokens:
        num_characters = num_characters + len(token)

    if verbose :        
        print(f"There are {num_tokens} tokens in the data.")
        print(f"There are {num_unique_tokens} unique tokens in the data.")
        print(f"There are {num_characters} characters in the data.")
        print(f"The lexical diversity is {lexical_diversity:.3f} in the data.")
    
        # print the five most common tokens
        # Pass the split_it list to instance of Counter class.
        count1 = Counter(tokens)

        # most_common() produces k frequently encountered
        # input values and their respective counts.
        most_occur = count1.most_common(5)
        print(most_occur)
  
    return([num_tokens, num_unique_tokens,
            lexical_diversity,
            num_characters])


    
def is_emoji(s):
    return(s in all_language_emojis)

def contains_emoji(s):
    
    s = str(s)
    emojis = [ch for ch in s if is_emoji(ch)]

    return(len(emojis) > 0)


def remove_stopwords(data):
    sw = stopwords.words("english")
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word.lower() not in sw:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    cleanedArray = list(filter(None, output_array))    
    cleanedArray = [x for x in cleanedArray if str(x) != 'nan']
    return cleanedArray
 
def remove_punctuation(text, punct_set=tw_punct) : 
    #punctuation = set(punctuation) # speeds up comparison
    punct_set = set(punct_set)
    return("".join([ch for ch in text if ch not in punct_set]))

def tokenize(text) : 
    """ Splitting on whitespace rather than the book's tokenize function. That 
        function will drop tokens like '#hashtag' or '2A', which we need for Twitter. """
    
    # modify this function to return tokens
    #is this function needed? Why don't we just use string.split()?
    #also removing "nulls"
    return( re.split('\s+', text) )

def prepare(text, pipeline) : 
    tokens = str(text)
    
    for transform in pipeline : 
        tokens = transform(tokens)
        
    return(tokens)


## Data Ingestion

Use this section to ingest your data into the data structures you plan to use. Typically this will be a dictionary or a pandas DataFrame.

In [4]:
# Feel fre to use the below cells as an example or read in the data in a way you prefer

data_location = "C:/Users/elfek/datamining/" # change to your location if it is not in the same directory as your notebook
twitter_folder = "twitter/"
lyrics_folder = "lyrics/"
artist1 = 'glassanimals'
artist2 = 'joyner'
artist_files = {artist1:'cher_followers_data.txt',
                artist2:'robynkonichiwa_followers_data.txt'}


In [5]:
twitter_data = pd.read_csv(data_location + twitter_folder + artist_files[artist1],
                           sep="\t",
                           quoting=3)

twitter_data['artist'] = artist1
twitter_data_2 = pd.read_csv(data_location + twitter_folder + artist_files[artist2],
                             sep="\t",
                             quoting=3)
twitter_data_2['artist'] = artist2

twitter_data = pd.concat([
    twitter_data,twitter_data_2])
    
del(twitter_data_2)

In [6]:
# read in the lyrics here


#reading the file
lyrics1 = []  
os.chdir(r"C:\Users\elfek\datamining\lyrics")
for filename in os.listdir(artist1):
    with open(os.path.join(artist1, filename), 'r', encoding="utf-8") as f:
        songLyrics = f.read()
        lyrics1.append(songLyrics.split('\n', 1)[1:])

lyrics2 = []        
os.chdir(r"C:\Users\elfek\datamining\lyrics")
for filename in os.listdir(artist2):
    with open(os.path.join(artist2, filename), 'r', encoding="utf-8") as f:
        songLyrics = f.read()
        lyrics2.append(songLyrics.split('\n', 1)[1:]) #extend
        #print(songTitle)
        

lyrics_data =  pd.DataFrame(lyrics1)
lyrics_data['artist'] = artist1
lyrics_data_2 =  pd.DataFrame(lyrics2)
lyrics_data_2['artist'] = artist2

lyrics_data = pd.concat([
    lyrics_data,lyrics_data_2])
    
del(lyrics_data_2)

lyrics_data.rename(columns={0: "lyrics"}, inplace = True)

lyrics_data

,lyrics,artist
0,"\nGlass Animals Lyrics\n""Agnes""\nAgnes just st...",glassanimals
1,"\nGlass Animals Lyrics\n""Black Mambo""\nWhat'll...",glassanimals
2,"\nGlass Animals Lyrics\n""Cane Shuga""\nBaby, do...",glassanimals
3,"\nGlass Animals Lyrics\n""Cocoa Hooves""\nThis o...",glassanimals
4,"\nGlass Animals Lyrics\n""Cocoa Hooves, Pt. II""...",glassanimals
...,...,...
104,"\nJoyner Lucas & J. Cole Lyrics\n""Your Heart""\...",joyner
105,"\nJoyner Lucas Lyrics\n""Zeze Freestyle""\nAhh, ...",joyner
106,"\nJoyner Lucas Lyrics\n""Zim Zimma""\nZim Zimma ...",joyner
107,"\nJoyner Lucas & Meek Mill Lyrics\n""Run It""\n[...",joyner


## Tokenization and Normalization

In this next section, tokenize and normalize your data. We recommend the following cleaning. 

**Lyrics** 

* Remove song titles
* Casefold to lowercase
* Remove punctuation
* Split on whitespace
* Remove stopwords (optional)

Removal of stopwords is up to you. Your descriptive statistic comparison will be different if you include stopwords, though TF-IDF should still find interesting features for you.

**Twitter Descriptions** 

* Casefold to lowercase
* Remove punctuation other than emojis or hashtags
* Split on whitespace
* Remove stopwords

Removing stopwords seems sensible for the Twitter description data. Remember to leave in emojis and hashtags, since you analyze those. 

In [ ]:
# apply the `pipeline` techniques from BTAP Ch 1 or 5

my_pipeline = [str.lower, remove_punctuation, tokenize, remove_stopwords]

lyrics_data["tokens"] = lyrics_data["lyrics"].apply(prepare,pipeline=my_pipeline)
lyrics_data["num_tokens"] = lyrics_data["tokens"].map(len) 

twitter_data["tokens"] = twitter_data["description"].apply(prepare,pipeline=my_pipeline)
twitter_data["num_tokens"] = twitter_data["tokens"].map(len) 


In [ ]:
twitter_data['has_emoji'] = twitter_data["description"].apply(contains_emoji)

Let's take a quick look at some descriptions with emojis.

In [ ]:
twitter_data[twitter_data.has_emoji].sample(10)[["artist","description","tokens"]]

With the data processed, we can now start work on the assignment questions. 

Q: What is one area of improvement to your tokenization that you could theoretically carry out? (No need to actually do it; let's not make perfect the enemy of good enough.)

A: We can add other characters such as periods, commas, and forward slashes in addition to the space when extracting tokens. Every sentence at least has a word ending in a period.

## Calculate descriptive statistics on the two sets of lyrics and compare the results. 


In [ ]:
# your code here
#Aggregate the lyrics tokens by artist to be able to get the descriptive statistics.
tmpLyricList1 = []
for line in lyrics_data.loc[lyrics_data['artist'] == artist1]['tokens']:
    tmpLyricList1.extend(line)
    
tmpLyricList2 = []
for line in lyrics_data.loc[lyrics_data['artist'] == artist2]['tokens']:
    tmpLyricList2.extend(line)
    
#Aggregate the twitter description tokens by artist.
tmpTwitList1 = []
for line in twitter_data.loc[twitter_data['artist'] == artist1]['tokens']:
    tmpTwitList1.extend(line)

tmpTwitList2 = []
for line in twitter_data.loc[twitter_data['artist'] == artist2]['tokens']:
    tmpTwitList2.extend(line)
    
print("Lyrics Stats for " + artist1 + "\n" )
print(descriptive_stats(tmpLyricList1) )
print("\nLyrics Stats for " + artist2 + "\n" )
print(descriptive_stats(tmpLyricList2) )

#Twitter stats not needed
# print("\nTwitter Stats for " + artist1 + "\n" )
# print(descriptive_stats(tmpTwitList1) )
# print("\nTwitter Stats for " + artist1 + "\n" )
# print(descriptive_stats(tmpTwitList2) )

Q: what observations do you make about these data? 

A: Although there are double the number of lyrics for Cher compared to Robyn the stats are very similar. The same lexical diversity and the same top words although the ordering is slightly diff.


## Find tokens uniquely related to a corpus

Typically we would use TF-IDF to find unique tokens in documents. Unfortunately, we either have too few documents, if we view each data source as a single document, or too many, if we view each description as a separate document. In the latter case, our problem will be that descriptions tend to be short, so our matrix would be too sparse to support analysis. 

To get around this, we find tokens for each corpus that match the following criteria:

1. The token appears at least `n` times in all corpora
1. The tokens are in the top 10 for the highest ratio of appearances in a given corpora vs appearances in other corpora.

You will choose a cutoff for yourself based on the side of the corpus you're working with. If you're working with the Robyn-Cher corpora provided, `n=5` seems to perform reasonably well.


Term Frequency (TF)
The number of times a word appears in a document divded by the total number of words in the document. Every document has its own term frequency.

In [ ]:
#these veriations return a dict or a paired list of words and count.
def findWords1(bagOfWords, n):
    #sorted list of word counts
    #wordCountSorted = sorted((Counter(tmpTwitList1).most_common()).items(), key=lambda x : x[1], reverse=True)
    wordCountSorted = dict(Counter(tmpTwitList1).most_common())
    nCounted = {}
    for key, value in wordCountSorted.items():
        if (value >= n):
            nCounted[key] = value
    return nCounted
# def findWords2(bagOfWords, n):
#     #sorted list of word counts
#     #wordCountSorted = sorted((Counter(tmpTwitList1).most_common()).items(), key=lambda x : x[1], reverse=True)
#     wordCountSorted = Counter(tmpTwitList1).most_common()
#     nCounted = []
#     for key, value in wordCountSorted:
#         if (value >= n):
#             nCounted.append([key, value])
#     return nCounted

#Find token that occurs at least n times in each corpora.
#This only returns the words/tokens
def findWords3(bagOfWords, n):
    #sorted list of word counts
    #wordCountSorted = sorted((Counter(tmpTwitList1).most_common()).items(), key=lambda x : x[1], reverse=True)
    wordCountSorted = Counter(tmpTwitList1).most_common()
    nCounted = []
    for word, count in wordCountSorted:
        if (count >= n):
            nCounted.append(word)
    return nCounted

dictnWordsTwit1 = findWords1(tmpTwitList1, 5)
dictnWordsTwit2 = findWords1(tmpTwitList2, 5)
dictnWordsLyrics1 = findWords1(tmpLyricList1, 5)
dictnWordsLyrics2 = findWords1(tmpLyricList2, 5)

nWordsTwit1 = findWords3(tmpTwitList1, 5)
nWordsTwit2 = findWords3(tmpTwitList2, 5)
nWordsLyrics1 = findWords3(tmpLyricList1, 5)
nWordsLyrics2 = findWords3(tmpLyricList2, 5)

#Get a list of words with >n counts that exist in both corpora
#shared_keys = dict_of_dicts_a.keys() & dict_of_dicts_b.keys() # how to combine keys, but not needed
uniqueWordsTwitter = set(tmpTwitList1).intersection(set(tmpTwitList2))
uniqueWordsLyrics =set(tmpLyricList1).intersection(set(tmpLyricList2))

#show top and bottom 10 word counts
#This was a test to make sure only words counted n re more times are saved
nWordsTwit1 = findWords2(tmpTwitList1, 5)
nWordsTwit1[0:9] ,nWordsTwit1[-11:-1] , len(uniqueWordsTwitter), len(uniqueWordsLyrics)

In [ ]:
#Each corpus is a collection of documents. One corpus for all the lyrics for arist1, artist2... Corpora is plural of corporus
    
def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

#get frequency of words with >n for artist1's twitter corpora
tfTwitterArtist1 = computeTF(dict(Counter(tmpTwitList1).most_common()), tmpTwitList1 )
#get frequency of words with >n for artist1's lyrics corpora
tfLyricsArtist1 = computeTF(dict(Counter(tmpLyricList1).most_common()),  tmpLyricList1)
#get frequency of words with >n for artist1's twitter corpora
tfTwitterArtist2 = computeTF(dict(Counter(tmpTwitList2).most_common()), tmpTwitList2 )
#get frequency of words with >n for artist1's lyrics corpora
tfLyricsArtist2 = computeTF(dict(Counter(tmpLyricList2).most_common()),  tmpLyricList2)
#tfTwitterArtist1, tfLyricsArtist1, tfTwitterArtist2, tfLyricsArtist2


In [ ]:
print("Top freq tokens in twitter descriptions for " + artist1 + "\n" )
print(sorted(tfTwitterArtist1.items(), key=lambda x : x[1], reverse=True)[0:10])
print("\nTop freq tokens in twitter descriptions for " + artist2 + "\n" )
print(sorted(tfTwitterArtist2.items(), key=lambda x : x[1], reverse=True)[0:10])
print("\nTop freq tokens in lyrics for " + artist1 + "\n" )
print(sorted(tfLyricsArtist1.items(), key=lambda x : x[1], reverse=True)[0:10])
print("\nTop freq tokens in lyrics for " + artist2 + "\n" )
print(sorted(tfLyricsArtist2.items(), key=lambda x : x[1], reverse=True)[0:10])

Q: What are some observations about the top tokens? Do you notice any interesting items on the list? 

A: This provides more insight to the differences between the corpora than the lexical complexity figures. I can see there are some terms that exist in both artists' corpora. I was surprized that a bullet point emoji made it to the top 10 for robyn's followers. I wonder if that has a special meaning to her fans.

The first time only used the tokens that existed in both lists that had more than n = 5 occurances for each corpus. This caused all the top 10 words to be the same (but with difference frequencies). 

## Build word clouds for all four corpora. 

For building wordclouds, we'll follow exactly the code of the text. The code in this section can be found [here](https://github.com/blueprints-for-text-analytics-python/blueprints-text/blob/master/ch01/First_Insights.ipynb). If you haven't already, you should absolutely clone the repository that accompanies the book. 


In [ ]:
from matplotlib import pyplot as plt

def wordcloud(word_freq, title=None, max_words=200, stopwords=None):

    wc = WordCloud(width=800, height=400, 
                   background_color= "black", colormap="Paired", 
                   max_font_size=150, max_words=max_words)
    
    # convert data frame into dict
    if type(word_freq) == pd.Series:
        counter = Counter(word_freq.fillna(0).to_dict())
    else:
        counter = word_freq

    # filter stop words in frequency counter
    if stopwords is not None:
        counter = {token:freq for (token, freq) in counter.items() 
                              if token not in stopwords}
    wc.generate_from_frequencies(counter)
 
    plt.title(title) 

    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    
    
def count_words(df, column='tokens', preprocess=None, min_freq=2):

    # process tokens and update counter
    def update(doc):
        tokens = doc if preprocess is None else preprocess(doc)
        counter.update(tokens)

    # create counter and run through all data
    counter = Counter()
    df[column].map(update)

    # transform counter into data frame
    freq_df = pd.DataFrame.from_dict(counter, orient='index', columns=['freq'])
    freq_df = freq_df.query('freq >= @min_freq')
    freq_df.index.name = 'token'
    
    return freq_df.sort_values('freq', ascending=False)

#All four corpora
# tmpTwitList1
# tmpTwitList2
# tmpLyricList1
# tmpLyricList2



In [ ]:
plt.figure(figsize=(12,10))
plt.subplot(2,2,1)
wordcloud(count_words(lyrics_data.loc[lyrics_data['artist'] == artist1])['freq'], max_words=50)
plt.subplot(2,2,2)
wordcloud(count_words(twitter_data.loc[twitter_data['artist'] == artist1])['freq'], max_words=50)
plt.subplot(2,2,3)
wordcloud(count_words(lyrics_data.loc[lyrics_data['artist'] == artist2])['freq'], max_words=50)
plt.subplot(2,2,4)
wordcloud(count_words(twitter_data.loc[twitter_data['artist'] == artist2])['freq'], max_words=50)

Q: What observations do you have about these (relatively straightforward) wordclouds? 

A: A lot of common words and frequencies of such words accross the different artists' corpora. The difference mainly is the ranking of those common words in relation to each other.